In [1]:
import pandas as pd
import yfinance as yf
import datetime
import cohere
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# Cohere API Key
COHERE_API_KEY = "jJvToSpBrN0EHfiCsa0wNg3DRTfKqcQxbFwKRza9"

In [2]:
def get_current_stock_price(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.history(period='1d')['Close'].iloc[-1]
    except Exception as e:
        print("Error fetching stock price:", e)
        return None

In [3]:
def download_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, progress=False)

In [4]:
# Inputs
ticker = 'SPY'  # Default stock symbol
today = datetime.date.today()
duration_days = 3000
start_date = today - datetime.timedelta(days=duration_days)
end_date = today

# Download data
data = download_data(ticker, start_date, end_date)
data.tail()

YF.download() has changed argument auto_adjust default to True



1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,


In [5]:
# Initialize scaler
scaler = StandardScaler()

# Model training and prediction function
def model_engine(model, num_days_forecast):
    df = data[['Close']].copy()
    df['preds'] = df.Close.shift(-num_days_forecast)
    
    x = scaler.fit_transform(df.drop(['preds'], axis=1).values)
    y = df.preds[:-num_days_forecast].values
    x_train, x_test, y_train, y_test = train_test_split(x[:-num_days_forecast], y, test_size=0.2, random_state=7)
    
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    print(f'R2 Score: {r2_score(y_test, preds):.4f}')
    print(f'MAE: {mean_absolute_error(y_test, preds):.4f}')
    
    forecast_pred = model.predict(x[-num_days_forecast:])
    for i, pred in enumerate(forecast_pred, 1):
        print(f'Day {i}: ${pred:.2f}')

# Model choices
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Extra Trees': ExtraTreesRegressor(),
    'K-Neighbors': KNeighborsRegressor(),
    'XGBoost': XGBRegressor()
}

# Choose model
model_choice = 'Linear Regression'  # Change as needed
num_days_forecast = 5  # Forecast for 5 days
model_engine(models[model_choice], num_days_forecast)

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

In [ ]:
# Cohere LLM Chatbot Simulation
def chatbot_response(user_input):
    if not COHERE_API_KEY:
        print("Cohere API key not found.")
        return
    
    try:
        co = cohere.Client(COHERE_API_KEY)
        response = co.chat(message=user_input)
        print("Chatbot Response:", response.text)
    except Exception as e:
        print(f"Error from Cohere: {e}")

# Example chatbot interaction
user_question = "Tell me about the stock market today."
chatbot_response(user_question)